# Recommendation system
AIFFEL 대전 탐색 노드 8   
영화 추천 시스템 구축 프로젝트이다. 모델은 MF를 사용한다.
  
프로세스는 다음과 같다.     
1. Import package 
2. Data preprocessing 
3. EDA (basic)
4. Modeling
5. Analysis

## 1. Import package

In [1]:
import os
import pandas as pd
import numpy as np
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

## 2. Data preprocessing
- Data loading   
- Data filtering   
- Naming & indexing

In [2]:
# data loading
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# data filtering
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating to count (naming)
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
# indexing (movie to title)
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python',encoding = 'ISO-8859-1')
movie_to_title = {k:v for k,v in zip(movies['movie_id'], movies['title'])}
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 3. EDA
- count elements
- Popular movies
- Statistics

In [6]:
# EDA 
# movie count
print("movie count : ",ratings['movie_id'].nunique())

# user count
print("user count : ", ratings['user_id'].nunique())

# popular movie
movie_count = ratings.groupby('movie_id')['user_id'].count()
tmp = movie_count.sort_values(ascending=False).head(30)
for i,c in zip(tmp.index,tmp):
    print(i, movie_to_title[i],c)
print('---')
# 유저별 몇 명의 영화를 보고 있는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
print(user_count.describe())

# 유저별 play횟수 중앙값에 대한 통계
user_median = ratings.groupby('user_id')['count'].median()
print(user_median.describe())

# movie id (너무 옛날 영화라서 모르는 거 투성이다;;;)
my_favorite = [260 ,1196 ,1210 ,589 ,1] # star wars, terminator, and toy story

# 'ik'이라는 user_id가 위 영화를 1회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['ik']*5, 'movie_id': my_favorite, 'count':[1]*5})

if not ratings.isin({'user_id':['ik']})['user_id'].any():  # user_id에 'ik'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # check

movie count :  3628
user count :  6039
2858 American Beauty (1999) 3211
260 Star Wars: Episode IV - A New Hope (1977) 2910
1196 Star Wars: Episode V - The Empire Strikes Back (1980) 2885
1210 Star Wars: Episode VI - Return of the Jedi (1983) 2716
2028 Saving Private Ryan (1998) 2561
589 Terminator 2: Judgment Day (1991) 2509
593 Silence of the Lambs, The (1991) 2498
1198 Raiders of the Lost Ark (1981) 2473
1270 Back to the Future (1985) 2460
2571 Matrix, The (1999) 2434
480 Jurassic Park (1993) 2413
2762 Sixth Sense, The (1999) 2385
608 Fargo (1996) 2371
110 Braveheart (1995) 2314
1580 Men in Black (1997) 2297
527 Schindler's List (1993) 2257
1197 Princess Bride, The (1987) 2252
2396 Shakespeare in Love (1998) 2213
1617 L.A. Confidential (1997) 2210
318 Shawshank Redemption, The (1994) 2194
858 Godfather, The (1972) 2167
1265 Groundhog Day (1993) 2121
1097 E.T. the Extra-Terrestrial (1982) 2102
2997 Being John Malkovich (1999) 2066
2716 Ghostbusters (1984) 2051
296 Pulp Fiction (1994) 

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,ik,260,1,NaN
1,ik,1196,1,NaN
2,ik,1210,1,NaN
3,ik,589,1,NaN
4,ik,1,1,NaN


### indexing 

In [7]:
# indexing (추가된 데이터 고려)
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # ratings['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

ratings # check

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,count,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
0,6039,44,1,NaN
1,6039,117,1,NaN
2,6039,64,1,NaN
3,6039,92,1,NaN


## 4. Modeling
- Make CSR
- Fitting model   


In [8]:
# make CSR
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()
print(num_user, num_movie)
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

6040 3628


<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [9]:
#model
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
# model fitting
als_model.fit(csr_data_transpose)

# my_favorite = [260 ,1196 ,1210 ,589 ,1] # star wars, terminator, and toy story
ik, ss = user_to_idx['ik'], movie_to_idx[260] # star wars 4
ik_vector, ss_vector = als_model.user_factors[ik], als_model.item_factors[ss]

#print(ik_vector)
#print(ss_vector)
print(np.dot(ik_vector, ss_vector))

toy = movie_to_idx[1] # toy story
toy_vector = als_model.item_factors[toy]
np.dot(ik_vector, toy_vector)

  0%|          | 0/15 [00:00<?, ?it/s]

0.43044528


0.15097111

## 5. Analysis
- 비슷한 영화 추천
- 사용자 영화 추천
- 추천 이유

In [10]:
# recommendation
# movie_to_idx 를 뒤집어, index로부터 movie를 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

def get_similar_movie(movie: int):
    movie_id = movie_to_idx[movie]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    similar_movie = [movie_to_title[i] for i in similar_movie]
    return similar_movie

# similar movie
print(*get_similar_movie(1), sep = "\n") # toy story
print('---')
print(*get_similar_movie(260), sep = "\n") # star wars 4
print('---')


# user recommendation
user = user_to_idx['ik']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
#print(movie_recommended)
print(*[[movie_to_title[idx_to_movie[i[0]]],i[0]] for i in movie_recommended], sep = "\n")
print('---')

# contribution of recommendation
s = 60 # star wars 1
explain = als_model.explain(user, csr_data, itemid=s)

print(*[(movie_to_title[idx_to_movie[i[0]]], i[1]) for i in explain[1]], sep = "\n")

s = 50 # toy story 2
explain = als_model.explain(user, csr_data, itemid=s)

[(movie_to_title[idx_to_movie[i[0]]], i[1]) for i in explain[1]]

Toy Story (1995)
Toy Story 2 (1999)
Babe (1995)
Bug's Life, A (1998)
Aladdin (1992)
Groundhog Day (1993)
Lion King, The (1994)
Pleasantville (1998)
Beauty and the Beast (1991)
There's Something About Mary (1998)
---
Star Wars: Episode IV - A New Hope (1977)
Star Wars: Episode V - The Empire Strikes Back (1980)
Star Wars: Episode VI - Return of the Jedi (1983)
Raiders of the Lost Ark (1981)
Star Wars: Episode I - The Phantom Menace (1999)
Alien (1979)
Terminator, The (1984)
Back to the Future (1985)
Matrix, The (1999)
E.T. the Extra-Terrestrial (1982)
---
['Raiders of the Lost Ark (1981)', 120]
['Star Wars: Episode I - The Phantom Menace (1999)', 60]
['Matrix, The (1999)', 124]
['Terminator, The (1984)', 200]
['Back to the Future (1985)', 22]
['Alien (1979)', 193]
['Aliens (1986)', 651]
['Toy Story 2 (1999)', 50]
['Saving Private Ryan (1998)', 48]
['E.T. the Extra-Terrestrial (1982)', 26]
['Princess Bride, The (1987)', 5]
['Men in Black (1997)', 175]
['Jurassic Park (1993)', 107]
['Amer

[('Toy Story (1995)', 0.11788735085147736),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', 0.010472032242305033),
 ('Terminator 2: Judgment Day (1991)', 0.008990118394499955),
 ('Star Wars: Episode V - The Empire Strikes Back (1980)',
  -0.0001567037350512293),
 ('Star Wars: Episode IV - A New Hope (1977)', -0.0032338575004448415)]

### 결과에 대한 분석
위 코드의 첫 번째 출력(get_similar_movie(1))과 두 번째 출력(get_similar_movie(260)은 각각 toy story와 star wars 4와 유사한 영화 리스트를 출력하는 것이다. 대충 봤을 때, 첫 번째는 애니메이션 위주의 영화 리스트이고 두 번째는 공상과학 위주의 영화 리스트라서 어느 정도 유사한 영화 리스트를 보여주는 것을 알 수 있다.   
세 번째 출력(movie_recommended)의 경우는 가상의 유저('ik')의 영화 추천 리스트이다.(가상 유저의 본 영화 목록은 네번째(혹은 다섯 번째) 출력과 같다) 아마도 가상 유저가 star wars를 3편이나 본 것 때문에 그와 관련된 영화(두 번째 출력)가 많이 추천되어 있음을 알 수 있다.   
네 번째와 다섯 번째 출력은 각각 가상 유저에게 추천한 영화를 추천한 이유를 설명해주는 출력이다. 정확히는 가상 유저가 본 영화 리스트에서 해당 영화를 추천한 기여도를 보여준다. Star wars 1의 경우, 가상 유저가 본 3편의 star wars가 크게 작용함을 알 수 있고, toy story 2의 경우는 가상 유저가 본 toy story가 가장 크게 작용했음을 확인할 수 있다.

## 회고 및 루브릭 평가

### 루브릭 평가 항목
1. CSR matrix가 정상적으로 만들어졌다.(사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.)
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.(사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.)
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.(MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.)

### 평가 항목에 대한 수행
1. '4. modeling'에서 CSR matrix을 정상적으로 만든 것을 확인할 수 있다.(사이즈도 유저수와 영화수와 같게 만들어졌다.)
2. '4. modeling'에서 사용자('ik')와 아이템(star wars4, toy story)의 내적을 각각 계산을 했고, 좀 더 관련성이 있는 star war4가 더 높은 수치를 보임으로써 내적 수치가 의미있다는 것을 알 수 있다.
3. '5. analysis'에서 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다. 자세한 사항은 '5. analysis'의 결과에 대한 분석을 보면 될 것 같다.

### 회고

노션에 공지된 꼭 포함이 되어야 할 점
- 이번 프로젝트에서 **어려웠던 점,**
- 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**.
- 루브릭 평가 지표를 맞추기 위해 **시도한 것들**.
- 만약에 루브릭 평가 관련 지표를 **달성 하지 못했을 때, 이유에 관한 추정**.
- **자기 다짐**

---
- **어려웠던 점**    
노드에서 작은 오류가 많아서 오류를 해결하는 데 시간이 좀 걸린 것 같다. ssac에서 미리 한 것이라 참고해서 금방 해결은 했다. 다른 어려웠던 점은 데이터가 오래되었다는 것이다. 영화 제작 연도를 보면 2000년도 전에 만들어진 것 뿐이다. 추천 시스템을 만드는 것이 주제인데 정작 데이터에 대한 의미를 알기 힘들게 데이터를 주는 이유를 잘 모르겠다. 프로젝트는 해야 해서 유명한 것만 추출해서 의미를 해석하긴 했지만 그 이상의 의미를 찾기 힘든 데이터를 주어서 별로였다.
---
- **알아낸 점**    
파이썬 문법인 리스트 내포를 바로 출력하게 되면, 리스트의 원소가 일렬로 연결되어 출력되어서 보기에 안 좋았다. 그래서 print 사용법에 대해서 조금 공부를 해서 원소마다 '\n' 구분자를 이용하여 출려하는 방법을 알아냈다.

- **모호한 점**    
추천 시스템에서는 데이터의 의미를 잘 아는 것이 중요한데 위에서 한 번 언급했듯이 너무 오래된 데이터를 주어서 의미를 이끌어 내기 어려운 것 같다. 노드를 만든 제작자가 어떤 의도로 이걸 해석하기를 바랬는지 모르겠다.(한 번 편집기로 열어서 영화 목록을 한 번 훍어 봤는데 대부분 모르는 것 밖에 없었다.) 
---
- **시도한 것들**   
처음에는 그냥 가상 유저('ik')의 영화 리스트를 처음 5개로 했는데 도저히 의미를 해석하기가 힘들었다. 그래서 그냥 유명한 영화 중에 시리즈가 있는 걸로 선택하여 의미를 해석했다.

---
- **우브릭 평가 관련 지표**   
제 예상에는 모두 달성되었다고 생각한다. 그 이유는 위에 있는 **평가 항목에 대한 수행**에 나와있다.  
- **자기 다짐** 및 **나의 생각들**      
이번 프로젝트도 역시 실제 모델을 만드는 것이 아닌 가져다 쓰는 것이라서 크게 오래 걸리지 않았다. 다만 크고 작은 오류들이 있어서 해결하는데 시간을 많이 보낸 것 같다. 그리고 의미를 해석하는 것이 더 어려웠다. 앞으로 노드에는 의도와 맞는 데이터를 제공해줬으면 좋겠다. 추천 시스템에 대해서 개인적으로 잠깐 공부한 적이 있는데 그래서 새롭거나 어려운 것들은 없었던 것 같다. 간단히 개념을 알고 있었는데, 이번 프로젝트에서 사용한 CSR matrix의 경우에는 한 번 자세히 공부해보는 것이 좋을 것 같다.